In [39]:
#Cargar la extensión autoreload
%load_ext autoreload
%autoreload 2
# Retriever tools
from libs.chatbot import Chatbot
from libs.vector_retriever import VectorRetriever
from libs.lexical_retriever import LexicalRetriever
from libs.basic_document_storage import BasicStorage
from libs.hybrid_retriever import HybridRetriever
from libs.pdf_loader import PDFLoader
from libs.basic_text_splitter import BasicTextSplitter

# Prompt Tools
from prompts.academic_assistant_prompt import system_prompt as academic_system_prompt
from prompts.academic_assistant_prompt import user_prompt as academic_user_prompt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# 1. Load, chunk and index the contents of the blog to create a retriever.
# Load the documents
pdf_loader = PDFLoader(file_filter={".pdf"})
documents = pdf_loader.load_data()
# Split the documents in chunks
splitter = BasicTextSplitter()
chunked_documents = splitter.split_documents(documents)
# Store the documents
basic_storage = BasicStorage()
basic_storage.add_documents(chunked_documents)


In [3]:
# 2. Create a retriever to handle the retrieval process.
# For handling the storage and retrieval of data chunks with semantic search
semantic_retriever = VectorRetriever()
# For handling the storage and retrieval of data chunks with lexical search
lexical_retriever = LexicalRetriever()
# A retriever for handling all the retrieval process
hybrid_retriever = HybridRetriever(storage_= basic_storage, retrievers=[semantic_retriever,lexical_retriever])
# Add the documents to the retriever
print("Adding documents to the retriever")
hybrid_retriever.add(chunked_documents)


Adding documents to the retriever


WARNING clustering 659 points to 25 centroids: please provide at least 975 training points


BM25S Create Vocab:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Convert tokens to indices:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/659 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/659 [00:00<?, ?it/s]

In [49]:
bot = Chatbot(hybrid_retriever,
              system_prompt=academic_system_prompt,
              user_prompt=academic_user_prompt
              )

In [54]:
query1 = "Cuales son los subsistemas endogenos del modelo pentagonal"

In [55]:
bot.reply(query1)

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

'Los subsistemas endógenos del modelo pentagonal son: recursos turísticos y atractivos, productos turísticos, organización y comercialización de la oferta turística, y flujos turísticos.'

In [52]:
query2 = "Menciona todos los subsistemas que encuentres en el contexto del modelo pentagonal"

In [53]:
bot.reply(query2)

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

'Subsistemas exógenos o integrantes del ‘entorno turístico’, Subsistemas endógenos o propiamente turísticos, Subsistema macroentorno o ambiente general'